# 2019-2023 Philadelphia Car Crash Data Preparation

This is the first of several workbooks where I will analyze auto accident data in the city of Philadelphia.

### What steps did I take to prepare the data?
#### Import crash data
- Import annual crash data sets
- Make sure each table has the same headings
- Combine annual tables into one table

#### Decode variables
- Manually scraped the data mappings from the [Data Dictionary](https://gis.penndot.gov/gishub/crashZip/Crash%20Data%20Dictionary%2005.2023.pdf)
- Import the decode mapping table
- Clean the decode mapping table
- Add decoded variables
- Look for unmapped values

### What does the full dataset look like?
The [whole dataset](https://pennshare.maps.arcgis.com/apps/webappviewer/index.html?id=8fdbf046e36e41649bbfd9d7dd7c7e7e) includes all counties in Pennsylvania, USA and, as of September 2024, covers the years 2001-2023.  There are other tables available with the following information:
- General Vehicle information
- Commercial Vehicles
- Motorcycle/Bicycles
- Flags for Alcohol/Drug/CellPhone Usage/Aggressive Driving etc.
- Persons involved in crashes
- Trailer crashes

I will pull these tables in for future iterations of this analysis.

## Import crash data

In [47]:
#https://pennshare.maps.arcgis.com/apps/webappviewer/index.html?id=8fdbf046e36e41649bbfd9d7dd7c7e7e
#https://gis.penndot.gov/gishub/crashZip/OPEN%20DATA%20PORTAL%20Database%20Primer%2010-16.pdf
#https://gis.penndot.gov/gishub/crashZip/Crash%20Data%20Dictionary%2005.2023.pdf

In [65]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
#import matplotlib.cm as cm
#import seaborn as sns
from glob import glob # for importing files with similar names using wildcard
import geopandas as gpd # GIS
from zipfile import ZipFile
import tempfile

pd.set_option("display.max_rows", None, "display.max_columns", None)

# set data paths and filenames
filename_decode_map = 'crash_data_dict.csv'

In [66]:
# Import annual crash data files and append them to a list of dataframes
list_of_dfs = []

# create folder to save unzipped files
unzip_path = f'''{os.getcwd()}\\unzipped_csv\\'''
os.makedirs(unzip_path, exist_ok = True)

# unzip all archives
for zip_path in glob('\\zip_files\\Philadelphia*.zip'):
    zf = ZipFile(zip_path)
    zf.extractall(unzip_path)

# import CSV files as dataframes
for csv_path in glob(f'''{unzip_path}CRASH*.csv'''): # just import crash files
    df = pd.read_csv(csv_path,low_memory=False)
    df['filename'] = csv_path.replace(unzip_path,'') # add column with file name so we can trace the source if I find bad data
    list_of_dfs.append(df)


In [67]:
# check if files have the same columns before combining
for i in range(len(list_of_dfs)-1):
    try:
        if ((list_of_dfs[i].columns == list_of_dfs[i+1].columns)-1).sum() == 0:
            print(f'''OK - {list_of_dfs[i].filename[0]} / {list_of_dfs[i+1].filename[0]} have same columns.''')
    except:
        print(f'''**** CHECK - {list_of_dfs[i].filename[0]} / {list_of_dfs[i+1].filename[0]} don't have same columns.''')
        earlier_extra_cols = set(list_of_dfs[i].columns)-set(list_of_dfs[i+1].columns)
        later_extra_cols = set(list_of_dfs[i+1].columns)-set(list_of_dfs[i].columns)
        if len(earlier_extra_cols) > 0:
            print(f'''******** {list_of_dfs[i].filename[0]} has extra column(s): {earlier_extra_cols}''')
        if len(later_extra_cols) > 0:
            print(f'''******** {list_of_dfs[i+1].filename[0]} has extra column(s): {later_extra_cols}''')

OK - CRASH_PHILADELPHIA_2019.csv / CRASH_PHILADELPHIA_2020.csv have same columns.
OK - CRASH_PHILADELPHIA_2020.csv / CRASH_PHILADELPHIA_2021.csv have same columns.
OK - CRASH_PHILADELPHIA_2021.csv / CRASH_PHILADELPHIA_2022.csv have same columns.
OK - CRASH_PHILADELPHIA_2022.csv / CRASH_PHILADELPHIA_2023.csv have same columns.


In [51]:
# combine dataframes into one
df_crash = pd.concat(list_of_dfs)

display(df_crash.shape)
display(df_crash.filename.value_counts())
df_crash.head()

(49235, 100)

CRASH_PHILADELPHIA_2019.csv    11159
CRASH_PHILADELPHIA_2021.csv    10552
CRASH_PHILADELPHIA_2020.csv    10171
CRASH_PHILADELPHIA_2022.csv     8783
CRASH_PHILADELPHIA_2023.csv     8570
Name: filename, dtype: int64

,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,CHLDPAS_DEATH_COUNT,CHLDPAS_SUSP_SERIOUS_INJ_COUNT,COLLISION_TYPE,COMM_VEH_COUNT,CONS_ZONE_SPD_LIM,COUNTY,CRASH_MONTH,CRASH_YEAR,DAY_OF_WEEK,DEC_LAT,DEC_LONG,DISPATCH_TM,DISTRICT,DRIVER_COUNT_16YR,DRIVER_COUNT_17YR,DRIVER_COUNT_18YR,DRIVER_COUNT_19YR,DRIVER_COUNT_20YR,DRIVER_COUNT_50_64YR,DRIVER_COUNT_65_74YR,DRIVER_COUNT_75PLUS,EST_HRS_CLOSED,FATAL_COUNT,HEAVY_TRUCK_COUNT,HORSE_BUGGY_COUNT,HOUR_OF_DAY,ILLUMINATION,INJURY_COUNT,INTERSECT_TYPE,INTERSECTION_RELATED,LANE_CLOSED,LATITUDE,LN_CLOSE_DIR,LOCATION_TYPE,LONGITUDE,MAX_SEVERITY_LEVEL,MCYCLE_DEATH_COUNT,MCYCLE_SUSP_SERIOUS_INJ_COUNT,MOTORCYCLE_COUNT,MUNICIPALITY,NONMOTR_COUNT,NONMOTR_DEATH_COUNT,NONMOTR_SUSP_SERIOUS_INJ_COUNT,NTFY_HIWY_MAINT,PED_COUNT,PED_DEATH_COUNT,PED_SUSP_SERIOUS_INJ_COUNT,PERSON_COUNT,POLICE_AGCY,POSSIBLE_INJ_COUNT,RDWY_SURF_TYPE_CD,RELATION_TO_ROAD,ROAD_CONDITION,ROADWAY_CLEARED,SCH_BUS_IND,SCH_ZONE_IND,SECONDARY_CRASH,SMALL_TRUCK_COUNT,SPEC_JURIS_CD,SUSP_MINOR_INJ_COUNT,SUSP_SERIOUS_INJ_COUNT,SUV_COUNT,TCD_FUNC_CD,TCD_TYPE,TFC_DETOUR_IND,TIME_OF_DAY,TOT_INJ_COUNT,TOTAL_UNITS,UNB_DEATH_COUNT,UNB_SUSP_SERIOUS_INJ_COUNT,UNBELTED_OCC_COUNT,UNK_INJ_DEG_COUNT,UNK_INJ_PER_COUNT,URBAN_RURAL,VAN_COUNT,VEHICLE_COUNT,WEATHER1,WEATHER2,WORK_ZONE_IND,WORK_ZONE_LOC,WORK_ZONE_TYPE,WORKERS_PRES,WZ_CLOSE_DETOUR,WZ_FLAGGER,WZ_LAW_OFFCR_IND,WZ_LN_CLOSURE,WZ_MOVING,WZ_OTHER,WZ_SHLDER_MDN,WZ_WORKERS_INJ_KILLED,filename
0,2020008819,NaN,2,0,0,0,0,0,0,0,0,4,0,NaN,67,3,2019,1,40.0214,-75.0794,NaN,6,0,0,0,0,0,0,0,0,NaN,0,0,0.0,8.0,1,1,1,NaN,0,40 01:17.161,NaN,1,75 04:45.952,3,0,0,0,67301,0,0,0,N,0,0,0,3,00000,0,NaN,1,1,NaN,N,N,N,0,NaN,1,0,0,0,0,NaN,820.0,1,2,0,0,0,0,0,2,0,2,3,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
1,2020007793,NaN,1,0,0,0,0,0,0,0,0,4,0,NaN,67,2,2019,6,40.0798,-75.0267,NaN,6,0,0,0,0,0,1,0,0,NaN,0,0,0.0,16.0,1,2,0,N,0,40 04:47.424,NaN,7,75 01:36.083,4,0,0,0,67301,0,0,0,N,0,0,0,3,00000,1,NaN,1,7,NaN,N,N,N,0,NaN,0,0,1,0,0,NaN,1600.0,2,2,0,0,0,1,0,2,0,2,10,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
2,2020007788,NaN,1,0,0,0,0,0,0,0,0,4,0,NaN,67,2,2019,5,39.9755,-75.1215,NaN,6,0,0,0,0,0,1,0,0,NaN,0,0,0.0,14.0,1,1,0,N,0,39 58:31.800,NaN,7,75 07:17.400,3,0,0,0,67301,0,0,0,N,0,0,0,2,00000,0,NaN,1,1,NaN,N,N,N,0,NaN,1,0,1,0,0,NaN,1445.0,1,2,0,0,0,0,0,2,0,2,3,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
3,2020008005,NaN,0,0,0,0,0,0,0,0,0,4,0,NaN,67,2,2019,6,39.9427,-75.2364,NaN,6,0,0,0,0,0,1,0,0,NaN,0,0,0.0,12.0,1,1,1,NaN,0,39 56:33.543,NaN,0,75 14:11.115,4,0,0,0,67301,0,0,0,N,0,0,0,2,00000,1,NaN,1,7,NaN,N,N,N,0,NaN,0,0,2,3,2,NaN,1253.0,1,2,0,0,0,0,0,2,0,2,10,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
4,2020006655,1250.0,1,0,0,0,0,0,0,0,0,7,0,NaN,67,5,2019,6,39.9870,-75.0878,1250.0,6,0,0,0,0,0,0,0,0,2.0,0,0,0.0,12.0,1,1,2,NaN,1,39 59:13.104,3.0,2,75 05:15.972,4,0,0,0,67301,0,0,0,N,0,0,0,1,67301,1,NaN,4,1,NaN,N,N,NaN,0,NaN,0,0,0,0,0,Y,1250.0,1,1,0,0,0,0,0,2,0,1,3,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv


## Decode variables

Several fields are encoded and they need to be decoded so that we can make charts, tables, maps that are easy to understand.  I will add several decoded columns (with the suffix "_dec" for "decoded").  I copy and pasted most of [PennDot's mapping table PDF](http://pennshare.maps.arcgis.com/sharing/rest/content/items/ffe20c6c3c594389b275c6772a281bcd/data) into a CSV named "crash_data_dict.csv".

#### What does encoded mean?
The crash table shows that a crash has "Road Condition" of "1".  What does "1" mean?  It stands for "Dry".  We want to see the words "Dry", "Snow", or "Wet" instead of the codes 1, 7 or 9.

### Import the decode mapping table

In [52]:
# Import the mapping table
df_decode_map = pd.read_csv('crash_data_dict.csv')
display(df_decode_map.shape)
df_decode_map.head()

(808, 2)

,Field,Code-Value
0,ACCESS_CTRL,1 - Limited Access
1,ACCESS_CTRL,2 - Partial Access
2,ACCESS_CTRL,3 - No Access Control
3,AIRBAG1,00 - Not Deployed
4,AIRBAG1,01 - Curtain


### Clean the decode mapping table

In [53]:
# List of columns that will be decoded
cols_to_decode = ['COLLISION_TYPE',
                  'COUNTY',
                  'DISTRICT',
                  'ILLUMINATION',
                  'LN_CLOSE_DIR',
                  'LOCATION_TYPE',
                  'MAX_SEVERITY_LEVEL',
                  'RDWY_SURF_TYPE_CD',
                  'RELATION_TO_ROAD',
                  'ROAD_CONDITION',
                  'SPEC_JURIS_CD',
                  'TCD_FUNC_CD',
                  'TCD_TYPE',
                  'WEATHER1',
                  'WEATHER2',
                  'WORK_ZONE_LOC',
                  'WORK_ZONE_TYPE']

# Keep rows for fields we want to decode
df_decode_map = df_decode_map[df_decode_map.Field.isin(cols_to_decode)]
display(df_decode_map.shape)
df_decode_map.head()

(205, 2)

,Field,Code-Value
136,COLLISION_TYPE,0 - Non-collision
137,COLLISION_TYPE,1 - Rear-end
138,COLLISION_TYPE,2 - Head-on
139,COLLISION_TYPE,3 - Backing
140,COLLISION_TYPE,4 - Angle


In [54]:
# Prepare decode_map by splitting 'Code-Value' at dash to create Code and Value columns
df_decode_map[['Code','Value']] = (df_decode_map['Code-Value'].str.split('-',1, expand=True))

# remove leading zeros from the codes and convert the code field to int
# for the variables that will be decoded, there are no instances of "0" and "00"
df_decode_map.Code = df_decode_map.Code.str.strip().astype(int)
df_decode_map.Value = df_decode_map.Value.str.strip()
df_decode_map.head()

,Field,Code-Value,Code,Value
136,COLLISION_TYPE,0 - Non-collision,0,Non-collision
137,COLLISION_TYPE,1 - Rear-end,1,Rear-end
138,COLLISION_TYPE,2 - Head-on,2,Head-on
139,COLLISION_TYPE,3 - Backing,3,Backing
140,COLLISION_TYPE,4 - Angle,4,Angle


In [55]:
# Unique codes in table
display(df_decode_map.Code.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 98, 99,  9, 10, 11, 12, 13, 14,
       15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
       66, 67])

In [56]:
#Column names and data types
df_crash.dtypes

CRN                                 int64
ARRIVAL_TM                        float64
AUTOMOBILE_COUNT                    int64
BELTED_DEATH_COUNT                  int64
BELTED_SUSP_SERIOUS_INJ_COUNT       int64
BICYCLE_COUNT                       int64
BICYCLE_DEATH_COUNT                 int64
BICYCLE_SUSP_SERIOUS_INJ_COUNT      int64
BUS_COUNT                           int64
CHLDPAS_DEATH_COUNT                 int64
CHLDPAS_SUSP_SERIOUS_INJ_COUNT      int64
COLLISION_TYPE                      int64
COMM_VEH_COUNT                      int64
CONS_ZONE_SPD_LIM                 float64
COUNTY                              int64
CRASH_MONTH                         int64
CRASH_YEAR                          int64
DAY_OF_WEEK                         int64
DEC_LAT                           float64
DEC_LONG                          float64
DISPATCH_TM                       float64
DISTRICT                            int64
DRIVER_COUNT_16YR                   int64
DRIVER_COUNT_17YR                 

In [57]:
# What does the combined data set look like now.
df_crash.head()

,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,CHLDPAS_DEATH_COUNT,CHLDPAS_SUSP_SERIOUS_INJ_COUNT,COLLISION_TYPE,COMM_VEH_COUNT,CONS_ZONE_SPD_LIM,COUNTY,CRASH_MONTH,CRASH_YEAR,DAY_OF_WEEK,DEC_LAT,DEC_LONG,DISPATCH_TM,DISTRICT,DRIVER_COUNT_16YR,DRIVER_COUNT_17YR,DRIVER_COUNT_18YR,DRIVER_COUNT_19YR,DRIVER_COUNT_20YR,DRIVER_COUNT_50_64YR,DRIVER_COUNT_65_74YR,DRIVER_COUNT_75PLUS,EST_HRS_CLOSED,FATAL_COUNT,HEAVY_TRUCK_COUNT,HORSE_BUGGY_COUNT,HOUR_OF_DAY,ILLUMINATION,INJURY_COUNT,INTERSECT_TYPE,INTERSECTION_RELATED,LANE_CLOSED,LATITUDE,LN_CLOSE_DIR,LOCATION_TYPE,LONGITUDE,MAX_SEVERITY_LEVEL,MCYCLE_DEATH_COUNT,MCYCLE_SUSP_SERIOUS_INJ_COUNT,MOTORCYCLE_COUNT,MUNICIPALITY,NONMOTR_COUNT,NONMOTR_DEATH_COUNT,NONMOTR_SUSP_SERIOUS_INJ_COUNT,NTFY_HIWY_MAINT,PED_COUNT,PED_DEATH_COUNT,PED_SUSP_SERIOUS_INJ_COUNT,PERSON_COUNT,POLICE_AGCY,POSSIBLE_INJ_COUNT,RDWY_SURF_TYPE_CD,RELATION_TO_ROAD,ROAD_CONDITION,ROADWAY_CLEARED,SCH_BUS_IND,SCH_ZONE_IND,SECONDARY_CRASH,SMALL_TRUCK_COUNT,SPEC_JURIS_CD,SUSP_MINOR_INJ_COUNT,SUSP_SERIOUS_INJ_COUNT,SUV_COUNT,TCD_FUNC_CD,TCD_TYPE,TFC_DETOUR_IND,TIME_OF_DAY,TOT_INJ_COUNT,TOTAL_UNITS,UNB_DEATH_COUNT,UNB_SUSP_SERIOUS_INJ_COUNT,UNBELTED_OCC_COUNT,UNK_INJ_DEG_COUNT,UNK_INJ_PER_COUNT,URBAN_RURAL,VAN_COUNT,VEHICLE_COUNT,WEATHER1,WEATHER2,WORK_ZONE_IND,WORK_ZONE_LOC,WORK_ZONE_TYPE,WORKERS_PRES,WZ_CLOSE_DETOUR,WZ_FLAGGER,WZ_LAW_OFFCR_IND,WZ_LN_CLOSURE,WZ_MOVING,WZ_OTHER,WZ_SHLDER_MDN,WZ_WORKERS_INJ_KILLED,filename
0,2020008819,NaN,2,0,0,0,0,0,0,0,0,4,0,NaN,67,3,2019,1,40.0214,-75.0794,NaN,6,0,0,0,0,0,0,0,0,NaN,0,0,0.0,8.0,1,1,1,NaN,0,40 01:17.161,NaN,1,75 04:45.952,3,0,0,0,67301,0,0,0,N,0,0,0,3,00000,0,NaN,1,1,NaN,N,N,N,0,NaN,1,0,0,0,0,NaN,820.0,1,2,0,0,0,0,0,2,0,2,3,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
1,2020007793,NaN,1,0,0,0,0,0,0,0,0,4,0,NaN,67,2,2019,6,40.0798,-75.0267,NaN,6,0,0,0,0,0,1,0,0,NaN,0,0,0.0,16.0,1,2,0,N,0,40 04:47.424,NaN,7,75 01:36.083,4,0,0,0,67301,0,0,0,N,0,0,0,3,00000,1,NaN,1,7,NaN,N,N,N,0,NaN,0,0,1,0,0,NaN,1600.0,2,2,0,0,0,1,0,2,0,2,10,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
2,2020007788,NaN,1,0,0,0,0,0,0,0,0,4,0,NaN,67,2,2019,5,39.9755,-75.1215,NaN,6,0,0,0,0,0,1,0,0,NaN,0,0,0.0,14.0,1,1,0,N,0,39 58:31.800,NaN,7,75 07:17.400,3,0,0,0,67301,0,0,0,N,0,0,0,2,00000,0,NaN,1,1,NaN,N,N,N,0,NaN,1,0,1,0,0,NaN,1445.0,1,2,0,0,0,0,0,2,0,2,3,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
3,2020008005,NaN,0,0,0,0,0,0,0,0,0,4,0,NaN,67,2,2019,6,39.9427,-75.2364,NaN,6,0,0,0,0,0,1,0,0,NaN,0,0,0.0,12.0,1,1,1,NaN,0,39 56:33.543,NaN,0,75 14:11.115,4,0,0,0,67301,0,0,0,N,0,0,0,2,00000,1,NaN,1,7,NaN,N,N,N,0,NaN,0,0,2,3,2,NaN,1253.0,1,2,0,0,0,0,0,2,0,2,10,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv
4,2020006655,1250.0,1,0,0,0,0,0,0,0,0,7,0,NaN,67,5,2019,6,39.9870,-75.0878,1250.0,6,0,0,0,0,0,0,0,0,2.0,0,0,0.0,12.0,1,1,2,NaN,1,39 59:13.104,3.0,2,75 05:15.972,4,0,0,0,67301,0,0,0,N,0,0,0,1,67301,1,NaN,4,1,NaN,N,N,NaN,0,NaN,0,0,0,0,0,Y,1250.0,1,1,0,0,0,0,0,2,0,1,3,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CRASH_PHILADELPHIA_2019.csv


### Add decoded variables

In [58]:
display(df_crash[cols_to_decode].dtypes)

COLLISION_TYPE          int64
COUNTY                  int64
DISTRICT                int64
ILLUMINATION            int64
LN_CLOSE_DIR          float64
LOCATION_TYPE           int64
MAX_SEVERITY_LEVEL      int64
RDWY_SURF_TYPE_CD     float64
RELATION_TO_ROAD        int64
ROAD_CONDITION          int64
SPEC_JURIS_CD         float64
TCD_FUNC_CD             int64
TCD_TYPE                int64
WEATHER1                int64
WEATHER2              float64
WORK_ZONE_LOC         float64
WORK_ZONE_TYPE        float64
dtype: object

In [59]:
# WEATHER2 is the only text column.  It has a value with several spaces that should be converted to null
df_crash.WEATHER2.unique()

array([nan,  3.,  9.,  5.,  7.,  4., 98.,  6., 10.,  8.,  2.,  1.])

In [60]:
#convert all encoded columns to Int64 data type
for col in cols_to_decode:
    if df_crash[col].dtype == 'object': # strip white space, replace blank value with np.nan and convert to Int64
        df_crash[col] = df_crash[col].str.strip().replace('','-1').astype('Int64').replace(-1,np.nan)
    if df_crash[col].dtype in ['int32', 'int64', 'float64']: # convert all numeric columns to to Int64
        df_crash[col] = df_crash[col].astype('Int64')

display(df_crash[cols_to_decode].dtypes)

COLLISION_TYPE        Int64
COUNTY                Int64
DISTRICT              Int64
ILLUMINATION          Int64
LN_CLOSE_DIR          Int64
LOCATION_TYPE         Int64
MAX_SEVERITY_LEVEL    Int64
RDWY_SURF_TYPE_CD     Int64
RELATION_TO_ROAD      Int64
ROAD_CONDITION        Int64
SPEC_JURIS_CD         Int64
TCD_FUNC_CD           Int64
TCD_TYPE              Int64
WEATHER1              Int64
WEATHER2              Int64
WORK_ZONE_LOC         Int64
WORK_ZONE_TYPE        Int64
dtype: object

In [61]:
# add decoded columns
for col in cols_to_decode:
    decode_dict = df_decode_map[df_decode_map.Field == col][['Code','Value']].set_index('Code').to_dict()['Value']
    df_crash[col + '_dec'] = df_crash[col].map(decode_dict)
    
    #check for missing values due to bad mapping
    display(df_crash[[col, col + '_dec', 'CRN']].groupby([col, col + '_dec'],dropna=False).count())

,,CRN
COLLISION_TYPE,COLLISION_TYPE_dec,
0,Non-collision,462
1,Rear-end,9983
2,Head-on,1790
3,Backing,242
4,Angle,15895
5,Sideswipe (same dir.),6595
6,Sideswipe (Opposite dir.),1053
7,Hit fixed object,6537
8,Hit pedestrian,6534


,,CRN
COUNTY,COUNTY_dec,
67,PHILADELPHIA,49235


,,CRN
DISTRICT,DISTRICT_dec,
6,"District 6 (Bucks, Chester, Delaware, Montgomery, Philadelphia Counties)",49235


,,CRN
ILLUMINATION,ILLUMINATION_dec,
1,Daylight,26673
2,Dark - no streetlights,580
3,Dark - streetlights,19761
4,Dusk,937
5,Dawn,711
6,Dark - unknown roadway lighting,276
8,Other,294
9,Unknown,3


,,CRN
LN_CLOSE_DIR,LN_CLOSE_DIR_dec,
1,North,3297
2,South,3321
3,East,2591
4,West,2575
5,North and South,841
6,East and West,889
7,"All (N,S,E,W)",674
<NA>,NaN,35047


...

,,CRN
TCD_TYPE,TCD_TYPE_dec,
0,Not applicable,26414
1,Flashing traffic signal,393
2,Traffic signal,15797
3,Stop sign,5642
4,Yield sign,121
5,Active RR crossing controls,13
6,Passive RR crossing controls,6
7,Police officer or flagman,38
8,Other Type TCD,529


,,CRN
WEATHER1,WEATHER1_dec,
2,Blowing Snow,23
3,Clear,39474
4,Cloudy,1752
5,"Fog, Smog, Smoke",88
6,Freezing Rain or Freezing Drizzle,102
7,Rain,6497
8,Severe Crosswinds,6
9,Sleet or Hail,45
10,Snow,411


,,CRN
WEATHER2,WEATHER2_dec,
1,"Blowing Sand, Soil, Dirt",5
2,Blowing Snow,25
3,Clear,19516
4,Cloudy,1973
5,"Fog, Smog, Smoke",120
6,Freezing Rain or Freezing Drizzle,59
7,Rain,2145
8,Severe Crosswinds,24
9,Sleet or Hail,42


,,CRN
WORK_ZONE_LOC,WORK_ZONE_LOC_dec,
1,Before the 1st work zone warning sign,9
2,Advance warning area,77
3,Transition area,28
4,Activity area,417
5,Termination area,17
8,Other,65
<NA>,NaN,48622


,,CRN
WORK_ZONE_TYPE,WORK_ZONE_TYPE_dec,
1,Construction,507
2,Maintenance,85
3,Utility company,8
8,Other,13
<NA>,NaN,48622


#### The mappings look good, so I will delete the encoded columns and rename the decoded columns

In [62]:
df_crash.drop(columns = cols_to_decode, inplace = True)
for col in cols_to_decode:
    df_crash.rename(columns = {col+'_dec':col}, inplace = True)

In [64]:
df_crash.columns

Index(['CRN', 'ARRIVAL_TM', 'AUTOMOBILE_COUNT', 'BELTED_DEATH_COUNT',
       'BELTED_SUSP_SERIOUS_INJ_COUNT', 'BICYCLE_COUNT', 'BICYCLE_DEATH_COUNT',
       'BICYCLE_SUSP_SERIOUS_INJ_COUNT', 'BUS_COUNT', 'CHLDPAS_DEATH_COUNT',
       'CHLDPAS_SUSP_SERIOUS_INJ_COUNT', 'COMM_VEH_COUNT', 'CONS_ZONE_SPD_LIM',
       'CRASH_MONTH', 'CRASH_YEAR', 'DAY_OF_WEEK', 'DEC_LAT', 'DEC_LONG',
       'DISPATCH_TM', 'DRIVER_COUNT_16YR', 'DRIVER_COUNT_17YR',
       'DRIVER_COUNT_18YR', 'DRIVER_COUNT_19YR', 'DRIVER_COUNT_20YR',
       'DRIVER_COUNT_50_64YR', 'DRIVER_COUNT_65_74YR', 'DRIVER_COUNT_75PLUS',
       'EST_HRS_CLOSED', 'FATAL_COUNT', 'HEAVY_TRUCK_COUNT',
       'HORSE_BUGGY_COUNT', 'HOUR_OF_DAY', 'INJURY_COUNT', 'INTERSECT_TYPE',
       'INTERSECTION_RELATED', 'LANE_CLOSED', 'LATITUDE', 'LONGITUDE',
       'MCYCLE_DEATH_COUNT', 'MCYCLE_SUSP_SERIOUS_INJ_COUNT',
       'MOTORCYCLE_COUNT', 'MUNICIPALITY', 'NONMOTR_COUNT',
       'NONMOTR_DEATH_COUNT', 'NONMOTR_SUSP_SERIOUS_INJ_COUNT',
       'NTFY

In [63]:
df_crash.to_csv('df_crash.csv')